## import libraries

In [4]:
import docx #to read docx files
import pandas as pd

In [5]:
import nltk #import Natural Language Toolkit to work with human language data
import string
exclude = set(string.punctuation) #punctuations that will be removed in the text
exclude.update("’", "´", "…", "–") #add "’", in some interviews this is used for it’s, it´s
exclude.remove('_') #I use '_' to connect words that should be taken together, e.g. 'natural gas'
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer() #converting words into their base form

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mann\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mann\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
import matplotlib.pyplot as plt #for plots, e.g. of wordclouds


Bad key text.latex.preview in file c:\users\mann\appdata\local\programs\python\python37-32\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file c:\users\mann\appdata\local\programs\python\python37-32\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file c:\users\mann\appdata\local\programs\python\python37-32\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 418 ('save

In [7]:
import re
from nltk.util import ngrams #to check for n-grams
import collections #to sort list by occurrences
from collections import Counter

In [8]:
#set size of text for other notebooks
var_text_size = 15

## import transcripts

In [9]:
#get the clean text from the other notebook
%run ./p_01_import_transcripts_paper.ipynb
# dict_all_interviews_qa: questions and answers combined. this dictionary contains all the interviews and their names (keys), e.g. 'UK_05_poli'

In [10]:
#prepare dataframe with all interviews & paragraphs & country & classification. 
#Afterwards: merge with the df_dominant_topic_country on the text. 
df_all_interviews = pd.DataFrame()
for y in dict_all_interviews_qa: #for each interview
    var_key_split = y.split('_')
    var_country = var_key_split[0]
    var_number = var_key_split[1]
    var_class = var_key_split[2]
    for z in range(0, len(dict_all_interviews_qa[y].paragraphs)):
        var_text = dict_all_interviews_qa[y].paragraphs[z].text
        df_all_interviews = df_all_interviews.append({'Country': var_country, 'no_interview':var_number, \
                                 'class':var_class, 'Text':var_text}, ignore_index=True)

In [11]:
len(df_all_interviews)

5392

## clean and process data

In [12]:
#turn the individual interviews into one long list, each paragraph is one element in the list

In [13]:
def get_doc_list_paras():
    doc_list_paras = []
    #dict_all_interviews = get_dict_all_interviews()
    for x in dict_all_interviews_qa:
        for y in dict_all_interviews_qa[x].paragraphs:
            var_paragraph_text = dict_all_interviews_qa[x].paragraphs[y].text
            if clean(var_paragraph_text) != '': #check if it has ONLY stopwords. If so, exclude!
                doc_list_paras = doc_list_paras+var_paragraph_text
    return(doc_list_paras)

In [14]:
#determine stopwords: stopwords are a list of words that are very very common but don’t provide useful information
stop = set(stopwords.words('english'))
#add words that are either overlapping between wordclouds/topics and/or irrelevant
stop |= set(['thats', 'whats', 'lets', 'would', 'wouldnt', 'could', 'couldnt', 'cannot', 'dont',\
             'yeah', 'yea', 'okay', 'yes', 'jaja', 'um', 'no', 'oh', 'ok',\
             'im', 'shes', 'hes', 'ive', 'ill', 'itll', 'youll', 'theyll', 'well', 'will', 'youre', 'theyre','u',\
             'theyd','cant', 'wont', 'doesnt', 'didnt', 'weve',\
             'think','know', 'going','say', 'get', 'want', 'come', 'go', 'see', 'said', 'put', 'let', 'got',\
             'tell', 'gonna','drove', 'getting', 'make', 'look', 'mean',\
             'lot', 'kind', 'there',  'also',  'one', 'course',  'like','actually', 'yet', 'issue',\
             'always', 'really', 'thing', 'obviously', 'even', 'maybe', 'something',  'already', '______',\
             'quite', 'anyway', 'blah', 'bla', 'fe', 'ta', 'ah', 'haha',\
            'qstart', 'qend'])  #'–' is in exclude!

In [15]:
# 'it' & 'there' is in stopwords, but still appears later in the topics. WHy?
# 'thats', 'that', 'and', "I'm" etc.! 
# -->I changed order of punc & stop and added words to stopwords above!
def clean(doc):
    try_punc_free = "".join(ch for ch in doc.lower() if ch not in exclude)
    try_stop_free = ' '.join([i for i in try_punc_free.split() if i not in stop])
    try_normalized = " ".join([lemma.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] \
                            else lemma.lemmatize(i) for i,j in nltk.pos_tag(nltk.word_tokenize(try_stop_free))])
    #old: try_normalized = " ".join(lemma.lemmatize(word) for word in try_stop_free.split()) # base or dictionary form of a word
    
    try_stop_free_2 = " ".join([i for i in try_normalized.split() if i not in stop]) #do again, because lemmatized words might be stop words
    return(try_stop_free_2)

In [16]:
#make a long list, each paragraph (document) is one item
def get_doc_list_paras():
    doc_list_paras = []
    for x in dict_all_interviews_qa:
        for y in range(0, len(dict_all_interviews_qa[x].paragraphs)):
            var_paragraph_text = dict_all_interviews_qa[x].paragraphs[y].text
            if clean(var_paragraph_text) != '': #check if it has ONLY stopwords
                doc_list_paras.append(var_paragraph_text)
    return(doc_list_paras)

In [17]:
doc_list_paras = get_doc_list_paras()

In [18]:
len(doc_list_paras)

5391

### check for n-grams

#### this part does not have to be run every time

In [19]:
#some words held meaning when they appear contiguously, like 'climate change' or 'ministry of energy'
#we want to keep those together by adding a '_', i.e. 'climate_change'
#can additionally use code from: https://nicharuc.github.io/topic_modeling/

#join all documents into one string
doc_one_string = doc_list_paras.copy()
doc_one_string = "".join(doc_one_string)

#create list with all n-grams for 2, 3, 4, 5 words
list_n = [2,3,4]
dict_lists_n_grams = {}
for x in list_n:
    var_str = doc_one_string.lower()
    var_str = re.sub(r'[^a-zA-Z0-9\var_str]', ' ', var_str) # exclude all non alphanumeric characters
    tokens = [token for token in var_str.split(" ") if token != ""]
    output = list(ngrams(tokens, x)) #3,4,5
    output_counter = Counter(output)
    #create list with the n-grasm sorted by appearance. Only if more than 25 appearances.
    dict_lists_n_grams[str(x)+'_grams'] = Counter({k: c for k, c in output_counter.items() if c >= 25}).most_common()
    # collections. xxx 
    #drop n-grams that appear less than 20 times
    #for y in dict_lists_n_grams:

#drop some n-grams from list
dict_lists_n_grams_nouns = {}
for x in dict_lists_n_grams:
    dict_lists_n_grams_nouns[x] = []
    list_NN = ['NN', 'NNS'] #JJ Adjective. #NN Noun, Singular. #NNS Noun Plural
    #for 2-grams: first word should be adjective or noun, second noun.
    if x == '2_grams':
        for y in range(0, len(dict_lists_n_grams[x])):
            words = dict_lists_n_grams[x][y][0]
            tag = nltk.pos_tag(words)
            if tag[0][1] in ['JJ', 'NN', 'NNS'] and tag[1][1] in list_NN:
                #drop words that are only 't','i' or 's'
                if 't' not in words and 'i' not in words and 's' not in words:
                    dict_lists_n_grams_nouns[x].append(dict_lists_n_grams[x][y])
    #for 3-grams: at least one word should be noun
    if x == '3_grams':
        for y in range(0, len(dict_lists_n_grams[x])):
            words = dict_lists_n_grams[x][y][0]
            tag = nltk.pos_tag(words)
            #first word should be adjective or noun, second noun
            if (tag[0][1] in list_NN or tag[1][1] in list_NN or tag[2][1] in list_NN):
                if 't' not in words and 'i' not in words and 's' not in words:
                    dict_lists_n_grams_nouns[x].append(dict_lists_n_grams[x][y])
    #for 4-grams:at least one word should be noun
    if x == '4_grams':
        for y in range(0, len(dict_lists_n_grams[x])):
            words = dict_lists_n_grams[x][y][0]
            tag = nltk.pos_tag(words)
            #first word should be adjective or noun, second noun
            if (tag[0][1] in list_NN or tag[1][1] in list_NN or tag[2][1] in list_NN or tag[3][1] in list_NN):
                if 't' not in words and 'i' not in words and 's' not in words:
                    dict_lists_n_grams_nouns[x].append(dict_lists_n_grams[x][y])

#### this part has to be run

In [20]:
#Manually check the entire lists and extract words
#from tetragrams
list_tetragrams = ['coal fired power plants', 'coal fired power plant', 'ministry of the environment',\
'carbon capture and storage', 'committee on climate change', \
'coal fired power stations', 'coal fired power station']

#form trigrams
list_trigrams = ['coal phase out', 'ministry of energy', 'point of view', 'coal power plant',\
'state owned enterprise', 'net zero target', 'ministry of finance', 'feed in tariff', \
'department of energy', 'ministry of economics', 'power purchase agreements', 'power purchase agreement', \
'coal exit act', 'coal power plants', 'coal power plant', 'security of supply','ministry of environment',\
'climate change act', 'ministry of mines', 'power development plan', 'access to electricity',\
'clean power plan', 'environmental impact assessment', 'renewable energy act', \
'national development plan', 'energy regulatory commission', 'independent power producers',\
'state owned enterprises', 'minister of energy', 'fridays for future']

#from bigrams
list_bigrams = ['renewable energy', 'climate change', 'phase out', 'power plant', 'power plants', 'natural gas',\
'energy sector', 'coal fired', 'net zero', 'coal power', 'coal plants', 'coal plant', 'coal commission',\
'energy efficiency', 'energy policy', 'carbon price', 'private sector', 'long term', 'climate protection',\
'prime minister', 'power stations', 'state owned', 'climate policy', 'power sector', 'carbon pricing',\
'united states', 'world bank', 'coal exit', 'low carbon', 'structural change', 'shut down',\
'energy transition', 'renewable energies', 'fossil fuel', 'energy act', 'fossil fuels', 'development plan',\
'civil society', 'energy mix', 'eu ets', 'energy security', 'power plan', 'emissions trading', 'energy system',\
'paris agreement', 'electricity generation', 'power station', 'electricity price', 'capacity market',\
'trade unions', 'electricity market', 'hambach forest', 'off grid', 'price floor', 'gas prices', 'phasing out',\
'carbon capture', 'clean coal', 'economic growth',\
'federal government', 'local government', 'power generation', 'coal mining', 'offshore wind', 'federal states',\
 'know how', 'electricity prices', 'conservative party', 'environmental associations', 'central government',\
 'economic development', 'national government', 'south africa', 'solar panels', 'political economy', \
'air pollution', 'electric vehicles', 'think tanks', 'think tank', 'greenhouse gas', 'carbon neutrality', 'air quality', \
'net metering', 'base load', 'just transition', 'least cost', 'price floor', 'trade union',\
'extinction rebellion']

dict_n_to_replace = {'4_grams':list_tetragrams, '3_grams':list_trigrams, '2_grams':list_bigrams}

In [21]:
#the words should be connected with a '_'. Some words however are plural and should be replace by singular.l
dict_n_lists_automatically = {}
for x in dict_n_to_replace:
    list_replacements = list()
    for y in dict_n_to_replace[x]:
        list_replacements.append(y.replace(' ', '_'))
        dict_n_lists_automatically[x] = list_replacements

In [22]:
#change manually, e.g. plural to singular!
list_tetragrams_replacements = ['coal_fired_power_plant', 'coal_fired_power_plant',\
 'ministry_of_the_environment', 'carbon_capture_and_storage', 'committee_on_climate_change',\
 'coal_fired_power_station', 'coal_fired_power_station']

list_trigrams_replacements = ['coal_phase_out', 'ministry_of_energy', 'point_of_view', 'coal_power_plant',\
 'state_owned_enterprise', 'net_zero_target', 'ministry_of_finance', 'feed_in_tariff', \
 'department_of_energy', 'ministry_of_economics', 'power_purchase_agreement', 'power_purchase_agreement',\
 'coal_exit_act', 'coal_power_plant', 'coal_power_plant', 'security_of_supply', 'ministry_of_environment',\
 'climate_change_act', 'ministry_of_mines', 'power_development_plan', 'access_to_electricity',\
 'clean_power_plan', 'environmental_impact_assessment', 'renewable_energy_act', 'national_development_plan',\
 'energy_regulatory_commission', 'independent_power_producers', 'state_owned_enterprise',\
 'minister_of_energy', 'fridays_for_future']

list_bigrams_replacements = ['renewable_energy', 'climate_change', 'phase_out', 'power_plant', 'power_plant',\
 'natural_gas', 'energy_sector', 'coal_fired', 'net_zero', 'coal_power', 'coal_plant', 'coal_plant',\
 'coal_commission', 'energy_efficiency', 'energy_policy', 'carbon_price', 'private_sector', 'long_term',\
 'climate_protection', 'prime_minister', 'power_station', 'state_owned', 'climate_policy', 'power_sector',\
 'carbon_pricing', 'united_states', 'world_bank', 'coal_exit', 'low_carbon', 'structural_change',\
 'shut_down', 'energy_transition', 'renewable_energy', 'fossil_fuel', 'energy_act', 'fossil_fuel',\
 'development_plan', 'civil_society', 'energy_mix', 'eu_ets', 'energy_security', 'power_plan',\
 'emissions_trading', 'energy_system', 'paris_agreement', 'electricity_generation', 'power_station',\
 'electricity_price', 'capacity_market', 'trade_union', 'electricity_market', 'hambach_forest',\
 'off_grid', 'price_floor', 'gas_price', 'phasing_out', 'carbon_capture', 'clean_coal', 'economic_growth',\
 'federal_government', 'local_government', 'power_generation', 'coal_mining', 'offshore_wind', 'federal_states',\
 'know_how', 'electricity_price', 'conservative_party', 'environmental_association', 'central_government',\
 'economic_development', 'national_government', 'south_africa', 'solar_panel', 'political_economy',\
 'air_pollution', 'electric_vehicles', 'think_tank', 'think_tank', 'greenhouse_gas', 'carbon_neutrality',\
 'air_quality', 'net_metering', 'base_load', 'just_transition', 'least_cost', 'price_floor', 'trade_union', \
 'extinction_rebellion']

dict_n_replacements = {'4_grams':list_tetragrams_replacements, '3_grams':list_trigrams_replacements, \
            '2_grams':list_bigrams_replacements}

In [23]:
#create dictionaries, the keys are the original words, the values the new combined words
dict_tetragrams = dict(zip(dict_n_to_replace['4_grams'], dict_n_replacements['4_grams']))
dict_trigrams = dict(zip(dict_n_to_replace['3_grams'], dict_n_replacements['3_grams']))
dict_bigrams = dict(zip(dict_n_to_replace['2_grams'], dict_n_replacements['2_grams']))
#create one list including all three
dict_list = [dict_tetragrams, dict_trigrams, dict_bigrams]

In [24]:
#replace the n-grams in each document:
doc_list_paras_n = doc_list_paras.copy()
for d in dict_list:
    for x in range(0, len(doc_list_paras_n)):
        for y in d.keys():
            if y in doc_list_paras_n[x].lower():
                doc_list_paras_n[x] = re.sub(y, d[y], doc_list_paras_n[x], flags=re.IGNORECASE)

### strip data

In [25]:
#remove the punctuations, stopwords and normalize the corpus
def get_doc_clean():
    doc_clean = [clean(doc).split() for doc in doc_list_paras_n]
    return(doc_clean)

In [26]:
doc_clean = get_doc_clean() #doc_clean contains n-grams!

In [29]:
set_countries = ['DE', 'UK', 'CL', 'CO', 'US', 'KE', 'ZA', 'ID', 'IN', 'VN', 'PH', 'PK'] #order from clustering

In [30]:
#for the wordclouds & high-frequency words:
#For each country: each reply (so each paragraph) is one text:
def get_dict_country_doc_list_paras_n_clean():
    #do one list per country (for wordcloud)
    dict_country_doc_list_paras = {}
    dict_country_doc_clean_list_paras = {}
    dict_country_doc_list_paras_n = {}
    dict_country_doc_list_paras_n_clean = {}
    for x in set_countries:
        #add all paragraphs into a list:
        dict_country_doc_list_paras[x] = []
        for y in dict_all_interviews_qa:
            if x in y: #if country abbreviation is in the interview title
                dict_country_doc_list_paras[x] = dict_country_doc_list_paras[x]+([p.text for p in dict_all_interviews_qa[y].paragraphs if p.text])
        #replace the n-grams in each document:
        dict_country_doc_list_paras_n[x] = dict_country_doc_list_paras[x].copy()
        for d in dict_list: #dict_list contains all the n-grams
            for n in range(0, len(dict_country_doc_list_paras_n[x])):
                for k in d.keys():
                    if k in dict_country_doc_list_paras_n[x][n].lower(): #if n-gram in there, replace!
                        dict_country_doc_list_paras_n[x][n] = re.sub(k, d[k], dict_country_doc_list_paras_n[x][n], flags=re.IGNORECASE)

        #clean the data
        dict_country_doc_list_paras_n_clean[x] = [clean(doc).split() for doc in dict_country_doc_list_paras_n[x]]
    return(dict_country_doc_list_paras_n_clean)

In [31]:
dict_country_doc_list_paras_n_clean = get_dict_country_doc_list_paras_n_clean()

## do versions with only long docs

In [104]:
list_wordcount_drop = [0, 1, 2, 5, 7, 10, 20, 50]

In [105]:
#first, need to link clean docs to original documents
#doc_list_paras contains all the original texts
#doc_clean all the cleaned documents, so only words (without stopwords etc.)
df_paras_and_clean = pd.DataFrame({'Text':doc_list_paras, 'Clean_words':doc_clean})

In [106]:
#count the words
#add column with the number of words ('num_words_clean)
df_paras_and_clean_count = df_paras_and_clean.copy()
for x in df_paras_and_clean_count.index:
    df_paras_and_clean_count.loc[x,'num_words_clean'] = \
            len(df_paras_and_clean_count.loc[x, 'Clean_words'])

In [107]:
#create doc_list_paras and doc_clean for each of the x (word limit to drop)
dict_doc_list_paras_x = {}
dict_clean_doc_x = {}
for x in list_wordcount_drop:
    
    list_paras_x = []
    for y in df_paras_and_clean_count.index:
        if df_paras_and_clean_count.loc[y, 'num_words_clean'] > x:
            list_paras_x.append(df_paras_and_clean_count.loc[y, 'Text'])
    dict_doc_list_paras_x['>'+str(x)] = list_paras_x
    
    list_x_docs = []
    for z in doc_clean:
        if len(z) > x :
            list_x_docs.append(z)
    dict_clean_doc_x['>'+str(x)] = list_x_docs